![JohnSnowLabs](https://sparknlp.org/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp/blob/master/examples/python/transformers/onnx/HuggingFace_ONNX_in_Spark_NLP_T5.ipynb)

## Import ONNX T5 models from HuggingFace 🤗 into Spark NLP 🚀

Let's keep in mind a few things before we start 😊

- ONNX support was introduced in  `Spark NLP 5.0.0`, enabling high performance inference for models.
- ONNX support for the `T5Transformer` is only available since in `Spark NLP 5.2.0` and after. So please make sure you have upgraded to the latest Spark NLP release
- You can import T5 models via `T5Model`. These models are usually under `Text2Text Generation` category and have `T5` in their labels
- This is a very computationally expensive module especially on larger sequence. The use of an accelerator such as GPU is recommended.
- Reference: [T5Model](https://huggingface.co/docs/transformers/model_doc/t5#transformers.T5Model)
- Some [example models](https://huggingface.co/models?other=T5)

## Export and Save HuggingFace model

- Let's install `transformers` package with the `onnx` extension and it's dependencies. You don't need `onnx` to be installed for Spark NLP, however, we need it to load and save models from HuggingFace.
- We lock `transformers` on version `4.52.3`. This doesn't mean it won't work with the future releases
- We will also need `sentencepiece` for tokenization.

In [ ]:
!pip install -q --upgrade transformers[onnx]==4.52.3 optimum onnx

- HuggingFace has an extension called Optimum which offers specialized model inference, including ONNX. We can use this to import and export ONNX models
- We'll use [google/flan-t5-base](https://huggingface.co/google/flan-t5-base) model from HuggingFace as an example
- In addition to `T5Model` we also need to save the tokenizer. This is the same for every model, these are assets needed for tokenization inside Spark NLP.
- If we want to optimize the model, a GPU will be needed. Make sure to select the correct runtime.

In [ ]:
MODEL_NAME = "google/flan-t5-base"
EXPORT_PATH = f"onnx_models/{MODEL_NAME}"

# Option 1: Export with optimization (O2)
# NOTE: May cause issues for T5-small or its variants due to a known ONNX Runtime bug.
# To fix it:
#  - Option A: Manually edit onnxruntime's source (not recommended in Colab)
#  - Option B: Skip optimization (recommended workaround)
# !optimum-cli export onnx --task text2text-generation-with-past --model $MODEL_NAME --optimize O2 $EXPORT_PATH

# Option 2: Export without optimization (safe for all models)
!optimum-cli export onnx --task text2text-generation-with-past --model $MODEL_NAME $EXPORT_PATH

Let's have a look inside these two directories and see what we are dealing with:

In [3]:
!ls -l {EXPORT_PATH}

total 2283468
-rw-r--r-- 1 root root      1515 Jun 16 22:01 config.json
-rw-r--r-- 1 root root 651215877 Jun 16 22:03 decoder_model_merged.onnx
-rw-r--r-- 1 root root 650868767 Jun 16 22:02 decoder_model.onnx
-rw-r--r-- 1 root root 594210416 Jun 16 22:02 decoder_with_past_model.onnx
-rw-r--r-- 1 root root 438701789 Jun 16 22:01 encoder_model.onnx
-rw-r--r-- 1 root root       142 Jun 16 22:01 generation_config.json
-rw-r--r-- 1 root root      2543 Jun 16 22:01 special_tokens_map.json
-rw-r--r-- 1 root root    791656 Jun 16 22:01 spiece.model
-rw-r--r-- 1 root root     20830 Jun 16 22:01 tokenizer_config.json
-rw-r--r-- 1 root root   2422234 Jun 16 22:01 tokenizer.json


- As you can see, we need to move the sentence piece models `spiece.model` from the tokenizer to assets folder which Spark NLP will look for

In [4]:
!mkdir -p {EXPORT_PATH}/assets && mv -t {EXPORT_PATH}/assets {EXPORT_PATH}/spiece.model

In [5]:
!ls -l {EXPORT_PATH}/assets

total 776
-rw-r--r-- 1 root root 791656 Jun 16 22:01 spiece.model


## Import and Save T5 in Spark NLP

- **Install and set up Spark NLP in Google Colab**
  - This example uses specific versions of `pyspark` and `spark-nlp` that have been tested with the transformer model to ensure everything runs smoothly.

In [6]:
!pip install -q pyspark==3.5.4 spark-nlp==5.5.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 635.7/635.7 kB 1.3 MB/s eta 0:00:00


Let's start Spark with Spark NLP included via our simple `start()` function

In [7]:
import sparknlp

spark = sparknlp.start()

print("Spark NLP version: ", sparknlp.version())
print("Apache Spark version: ", spark.version)

Spark NLP version:  5.5.3
Apache Spark version:  3.5.4


- Let's use `loadSavedModel` functon in `T5Transformer` which allows us to load the ONNX model
- Most params will be set automatically. They can also be set later after loading the model in `T5Transformer` during runtime, so don't worry about setting them now
- `loadSavedModel` accepts two params, first is the path to the exported model. The second is the SparkSession that is `spark` variable we previously started via `sparknlp.start()`
- NOTE: `loadSavedModel` accepts local paths in addition to distributed file systems such as `HDFS`, `S3`, `DBFS`, etc. This feature was introduced in Spark NLP 4.2.2 release. Keep in mind the best and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.st and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.

In [8]:
from sparknlp.annotator import T5Transformer

T5 = T5Transformer.loadSavedModel(EXPORT_PATH, spark)\
  .setUseCache(True) \
  .setTask("summarize:") \
  .setMaxOutputLength(200)

Let's save it on disk so it is easier to be moved around and also be used later via `.load` function

In [9]:
T5.write().overwrite().save(f"{MODEL_NAME}_spark_nlp")

Let's clean up stuff we don't need anymore

In [10]:
!rm -rf {EXPORT_PATH}

Awesome  😎 !

This is your ONNX T5 model from HuggingFace 🤗  loaded and saved by Spark NLP 🚀

In [11]:
! ls -l {MODEL_NAME}_spark_nlp

total 1065328
-rw-r--r-- 1 root root 651315411 Jun 16 22:12 decoder.onxx
-rw-r--r-- 1 root root 438768884 Jun 16 22:12 encoder.onxx
drwxr-xr-x 2 root root      4096 Jun 16 22:12 metadata
-rw-r--r-- 1 root root    791656 Jun 16 22:12 t5_spp


Now let's see how we can use it on other machines, clusters, or any place you wish to use your new and shiny T5 model 😊

In [12]:
from sparknlp.base import *
from sparknlp.annotator import *
from pyspark.ml import Pipeline

test_data = spark.createDataFrame([[
    """
    Transfer learning, where a model is first pre-trained on a data-rich task before being fine-tuned on a downstream task,
    has emerged as a powerful technique in natural language processing (NLP). The effectiveness of transfer learning has
    given rise to a diversity of approaches, methodology, and practice. In this paper, we explore the landscape of transfer
    learning techniques for NLP by introducing a unified framework that converts all text-based language problems into a
    text-to-text format. Our systematic study compares pre-training objectives, architectures, unlabeled data sets, transfer
    approaches, and other factors on dozens of language understanding tasks. By combining the insights from our exploration
    with scale and our new Colossal Clean Crawled Corpus, we achieve state-of-the-art results on many benchmarks covering
    summarization, question answering, text classification, and more. To facilitate future work on transfer learning for NLP,
    we release our data set, pre-trained models, and code.
    """
]], ["text"])

document_assembler = DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("document")

T5 = T5Transformer.load(f"{MODEL_NAME}_spark_nlp") \
    .setInputCols(["document"]) \
    .setOutputCol("summary")

pipeline = Pipeline().setStages([
    document_assembler,
    T5
])

result = pipeline.fit(test_data).transform(test_data)
result.select("summary.result").show(truncate=False)

+-----------------------------------------------------------------------------------------------------------+
|result                                                                                                     |
+-----------------------------------------------------------------------------------------------------------+
|[We introduce a unified framework that converts text-to-text language problems into a text-to-text format.]|
+-----------------------------------------------------------------------------------------------------------+



That's it! You can now go wild and use hundreds of T5 models from HuggingFace 🤗 in Spark NLP 🚀
